### Evaluate the motion artifacts of BOLD data
This measure indexes the movement of the head from one volume to the next, and is calculated as the sum of the absolute values of the differentiated realignment estimates (by backwards differences) at every timepoint.
<br>
You need to adjust the threshould for number of affected volumes and threshould for absolut value for FD.
 
-----------------------------------------------------------
Script written by Mehdi Behroozi
<br>
Biopsychology, 
<br>
Ruhr-University Bochum, Bochum, Germany
<br>
(2022.03)

-----------------------------------------------------------

In [ ]:
import glob
import os

nFD_threshold = 50 #number of affected volume which is alowded >> 20% of the data (edit here)
thresh = 0.9 #Absolute threshold value fo affected volume by motion (edit here)


data_path = '/mnt/d/Data/Pigeon/Sleep/analysis3'
 
bold_files = glob.glob('%s/sub*/*/bold.nii.gz'%(data_path))
for curr_bold in list(bold_files):
    print('Current data is: %s'%(curr_bold))
    cur_dir = os.path.join(os.path.dirname(curr_bold),'results')
    
    if os.path.isdir(cur_dir)==False:
        print('\t 1) Create results directory ...')
        os.makedirs(cur_dir)
    else:
        print('\t 1) resutls directory exists!!!')
        pass
     
    
    if os.path.isdir('%s/motion_assess'%(cur_dir))==False:
        print('\t 2) Creating %s/motion_assess directory ...'%(cur_dir))
        os.makedirs('%s/motion_assess'%(cur_dir))
    else:
        print('\t 2) %s/motion_assess directory exist! Deleting old and creating new directory ... '%(cur_dir))
        os.system('rm -r %s/motion_assess'%(cur_dir))
        os.makedirs('%s/motion_assess'%(cur_dir))
    
    
    print('\t 3) Runing fsl_motion_outliers with threshould of %s ... \n'%(thresh))
    os.system("fsl_motion_outliers -i {0} -o {1}/motion_assess/confound.txt --fd --thresh={2} -s {1}/motion_assess/fd_data.txt -p {1}/motion_assess/fd_plot -v > {1}/motion_assess/outlier_output.txt".format(curr_bold, cur_dir, thresh))

    # Last, if we're planning on modeling out scrubbed volumes later
    #   it is helpful to create an empty file if confound.txt isn't
    #   generated (i.e. no scrubbing needed).  It is basically a
    #   place holder to make future scripting easier
    if os.path.isfile("%s/motion_assess/confound.txt"%(cur_dir))==False:
        os.system("touch %s/motion_assess/confound.txt"%(cur_dir))



# Plot framewise displasment results
The results for single subjects will be ploted

In [ ]:
%matplotlib inline
%pylab inline

from IPython.display import display, Image
import glob
import os
data_path = '/mnt/d/Data/Pigeon/Sleep/analysis3'

FD_files = glob.glob('%s/sub*/*/results/motion_assess/fd_plot.png'%(data_path))

for curr_FD in FD_files:
    
    print(curr_FD.split('/')[-4]+'/'+curr_FD.split('/')[-3])
    plt.show()
    display(Image(filename=curr_FD))

# Plot Histogram
This code will pull FD data from all subjects and plot the histogram

In [ ]:
%matplotlib inline
%pylab inline
# import os
import glob
import matplotlib.pyplot as plt
import matplotlib
import os
data_path = '/mnt/d/Data/Pigeon/Sleep/analysis3'
FD_files = glob.glob('%s/sub*/*/results/motion_assess/fd_data.txt'%(data_path))
FD_data = []
for cur_FD in FD_files:
    #print(cur_FD)
    f = open(cur_FD, "r")
    for line in f:
        if line != " ":
            FD_data.append((float(line.rsplit()[0])))
            
plt.hist(FD_data, bins=50, orientation='horizontal')
plt.ylabel('framewise displacement (mm)')
plt.xlabel('# volume')
#plt.xlim([0,7000])
#plt.ylim([-.1,0.8])
plt.style.context('seaborn-bright')
#plt.savefig(os.path.join(data_path,'motion_parameters/FD.svg'), format='svg', dpi=1200)
#plt.savefig(os.path.join(data_path,'motion_parameters/FD.png'))


# Finding number of affected volumes for each run

In [ ]:
#or function version
def length_of_list(list_of_numbers, number):
    x = [i for i in list_of_numbers if i > number]
    return len(x)

In [ ]:
import glob
import matplotlib.pyplot as plt
import matplotlib
data_path = '/mnt/d/Data/Pigeon/Sleep/analysis2'

FD_files = glob.glob('%s/sub*/*/results/motion_assess/fd_data.txt'%(data_path))
thresh = 0.9 #Absolute threshold value fo affected volume by motion (edit here)
FD_data = []
for cur_FD in list(FD_files):
    FD_data = []
    
    #print(cur_FD)
    f = open(cur_FD, "r")
    for line in f:
        if line != " ":
            FD_data.append((float(line.rsplit()[0])))
    print([cur_FD.split('/')[-4],cur_FD.split('/')[-3]], ' \t --->> \t ', length_of_list(FD_data, thresh))# for task fmri it can be half of voxel size